## Connect Google Drive and GPU


In [1]:
%reset

# connect google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# connect colab gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Mounted at /content/drive
Sat Dec 18 03:16:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------

## Import Needed Libraries, Paramaters and Functions

In [4]:
import sys
import csv
import time
import os.path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils import prune
import torchvision
import matplotlib.pyplot as plt
from torch.utils.mobile_optimizer import optimize_for_mobile
from scipy import stats
from sklearn.utils import shuffle

SEED = 10
WINDOW_SIZE = 128
FEATURE_SIZE = 40
LABEL_SIZE = 12
BATCH_SIZE = 3
PATH = '/content/drive/MyDrive/CNNPaper'

In [5]:
def dataCleaning(dataCollection):
  dataCollection = dataCollection.drop(['handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4',
                      'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4',
                      'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'],
                      axis = 1)  # removal of orientation columns as they are not needed
  dataCollection = dataCollection.drop(dataCollection[dataCollection.activityID == 0].index) #removal of any row of activity 0 as it is transient activity which it is not used
  dataCollection = dataCollection.apply(pd.to_numeric, errors = 'coerce') #removal of non numeric data in cells
  dataCollection = dataCollection.interpolate() #removal of any remaining NaN value cells by constructing new data points in known set of data points
  dataCollection.reset_index(drop = True, inplace = True)
  for i in range(0,4):
    dataCollection["heartrate"].iloc[i]=100  
  return dataCollection

def read_data():
  if os.path.isfile(PATH + '/data/PAMAP2/data.cvs'):
    print("Start reading data ...")
    dataCollection = pd.read_csv(PATH + '/data/PAMAP2/data.cvs')
    print("Finish reading data ...")
  else:
    print("Start reading data ...")
    list_of_files = ['/data/PAMAP2/Protocol/subject101.dat',
              '/data/PAMAP2/Protocol/subject102.dat',
              '/data/PAMAP2/Protocol/subject103.dat',
              '/data/PAMAP2/Protocol/subject104.dat',
              '/data/PAMAP2/Protocol/subject105.dat',
              '/data/PAMAP2/Protocol/subject106.dat',
              '/data/PAMAP2/Protocol/subject107.dat',
              '/data/PAMAP2/Protocol/subject108.dat',
              '/data/PAMAP2/Protocol/subject109.dat' ]
    subjectID = [1,2,3,4,5,6,7,8,9]
    activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping'}

    colNames = ["timestamp", "activityID","heartrate"]

    IMUhand = ['handTemperature', 
          'handAcc16_1', 'handAcc16_2', 'handAcc16_3', 
          'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 
          'handGyro1', 'handGyro2', 'handGyro3', 
          'handMagne1', 'handMagne2', 'handMagne3',
          'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4']

    IMUchest = ['chestTemperature', 
          'chestAcc16_1', 'chestAcc16_2', 'chestAcc16_3', 
          'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3', 
          'chestGyro1', 'chestGyro2', 'chestGyro3', 
          'chestMagne1', 'chestMagne2', 'chestMagne3',
          'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4']

    IMUankle = ['ankleTemperature', 
          'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 
          'ankleAcc6_1', 'ankleAcc6_2', 'ankleAcc6_3', 
          'ankleGyro1', 'ankleGyro2', 'ankleGyro3', 
          'ankleMagne1', 'ankleMagne2', 'ankleMagne3',
          'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4']

    columns = colNames + IMUhand + IMUchest + IMUankle

    dataCollection = pd.DataFrame()
    for file in list_of_files:
        procData = pd.read_table(PATH + file, header=None, sep='\s+')
        procData.columns = columns
        procData['subject_id'] = int(file[-5])
        dataCollection = dataCollection.append(procData, ignore_index=True)

    dataCollection.reset_index(drop=True, inplace=True)
    dataCollection = dataCleaning(dataCollection)
    dataCollection.to_csv('data.cvs', index=False)
    print("Finish reading data ...")
  return dataCollection

def feature_normalize(data):
  """
    Normalize the feature data
    Paramater:
      data: a list of floats
    Return:
      a list of floats with normalized data
  """
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  return (data - mu) / sigma


def dataset_normalize(dataset):
  """
    Normalize the whole dataset
    Paramater:
      dataset: a DataFrame with the data and labels 
    Return:
      a DataFrame with the normalized data and labels 
  """
  dataset.dropna(axis=0, how='any', inplace=True)
  for col in dataset.columns:
    if col != 'timestamp' and col != 'activityID' and col != 'subject_id':
      dataset[col] = feature_normalize(dataset[col])
  return dataset

def windows(data, size):
  """
    Obatin the starting index and ending index according to window size
    Paramater:
      data: a list of floats
      size: int
    Return:
      Starting index, ending index
  """
  start = 0
  while start < data.count():
    yield int(start), int(start + size)
    start += (size / 2)

def dataset_segmentation(data):
  """
    Dataset segmentation according the window size
    Paramater:
      data: a list of floats
    Return:
      segments and labels 
  """
  print("Start segmentation with window size: ", WINDOW_SIZE)
  segments = np.empty((0, WINDOW_SIZE, FEATURE_SIZE))
  labels = np.empty((0))
  size = data['timestamp'].count()
  for (start, end) in windows(data['timestamp'], WINDOW_SIZE):
      temp = []
      for col in data.columns:
        if col != 'timestamp' and col != 'activityID' and col != 'subject_id':
          array = data[col][start:end].to_numpy().tolist()
          temp.append(array)
      if len(data["timestamp"][start:end]) == WINDOW_SIZE:
        segments = np.vstack([segments, np.dstack(temp)])
        labels = np.append(labels, stats.mode(data["activityID"][start:end])[0][0])
  labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
  segments = segments.reshape(len(segments), FEATURE_SIZE, WINDOW_SIZE)
  print("Finish segmentation ...")
  return segments, labels

def train_valid_test_split(segments, classes, test_x, test_y, k_fold):
  """
    Split train, valid and test datase
    Paramater:
      segments: a list of input data
      classes: a list of classes data
      k: k fold cross validation
    Return:
      segments and labels 
  """
  print("Start dataset split... ")
  seg_len = len(segments)
  idx_val = [0, int(seg_len/5*1), int(seg_len/5*2), int(seg_len/5*3), int(seg_len/5*4), seg_len]
  train_range1 = range(0, idx_val[k_fold])
  valid_range = range(idx_val[k_fold], idx_val[k_fold+1])
  train_range2 = range(idx_val[k_fold+1], seg_len)

  train_x = np.concatenate((segments[train_range1], segments[train_range2]), axis=0)
  train_y = np.concatenate((classes[train_range1], classes[train_range2]), axis=0)
  valid_x = segments[valid_range]
  valid_y = classes[valid_range]

  # get train data
  train_data = []
  for i in range(len(train_x)):
    train_data.append([train_x[i], train_y[i]])
  
  # get valid data
  valid_data = []
  for i in range(len(valid_x)):
    valid_data.append([valid_x[i], valid_y[i]])
  
  # get test data
  test_data = []
  for i in range(len(test_x)):
    test_data.append([test_x[i], test_y[i]])
  print(len(train_data))
  print(len(valid_data))
  print(len(test_data))

  # generate DataLoader for each dataset
  trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
  validloader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE)
  testloader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE)
  
  print("Finish dataset split... ")
  return trainloader, validloader, testloader


# Load and Save Train, Test, Valid Dataset





In [ ]:
TRAIN_LOADER_PATH = PATH + '/model/train_loader'
VALID_LOAER_PATH = PATH + '/model/valid_loader'
TEST_LOADER_PATH = PATH + '/model/test_loader'

dataset = dataset_normalize(read_data())
segments, classes = dataset_segmentation(dataset)

Start reading data ...
Finish reading data ...
Start segmentation with window size:  128
Finish segmentation ...


In [ ]:
np.random.seed(SEED)
segments, classes = shuffle(segments, classes)
test_x = segments[range(int(len(segments)*0.8), len(segments))]
test_y = classes[range(int(len(classes)*0.8), len(classes))]
total_x = segments[range(0, int(len(segments)*0.8))]
total_y = classes[range(0, int(len(classes)*0.8))]
print(len(test_x))
print(len(test_y))
print(len(total_x))
print(len(total_y))

cross_valid_range = 5

for k in range(cross_valid_range):
  print("Start spliting for k = " + str(k))
  trainloader, validloader, testloader = train_valid_test_split(total_x, total_y, test_x, test_y, k)
  CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
  CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
  CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
  torch.save(trainloader, CROSS_TRAIN_LOADER_PATH)
  torch.save(validloader, CROSS_VALID_LOADER_PATH)
  torch.save(testloader, CROSS_TEST_LOADER_PATH)
  print("Finish data loading...")

6072
6072
24284
24284
Start spliting for k = 0
Start dataset split... 
19428
4856
6072
Finish dataset split... 
Finish data loading...
Start spliting for k = 1
Start dataset split... 
19427
4857
6072
Finish dataset split... 
Finish data loading...
Start spliting for k = 2
Start dataset split... 
19427
4857
6072
Finish dataset split... 
Finish data loading...
Start spliting for k = 3
Start dataset split... 
19427
4857
6072
Finish dataset split... 
Finish data loading...
Start spliting for k = 4
Start dataset split... 
19427
4857
6072
Finish dataset split... 
Finish data loading...


# Load CNN Model and Other Helper Functions

In [17]:
NODE_SIZE = 128
KERNAL_SIZE = 10
LEARNING_RATE = 0.0002

k = 0

TRAIN_LOADER_PATH = PATH + '/model/train_loader'
VALID_LOAER_PATH = PATH + '/model/valid_loader'
TEST_LOADER_PATH = PATH + '/model/test_loader'
CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
trainloader = torch.load(CROSS_TRAIN_LOADER_PATH)
validloader = torch.load(CROSS_VALID_LOADER_PATH)
testloader = torch.load(CROSS_TEST_LOADER_PATH)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # Convolutional Layers
    self.features = nn.Sequential(
      nn.Conv1d(FEATURE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
    )
  
    self.fc1 = nn.Linear(NODE_SIZE*(WINDOW_SIZE-5*(KERNAL_SIZE-1)), 100)
    self.fc2 = nn.Linear(100, LABEL_SIZE)
    self.max = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.max(x)
    return x

def train_save_CNN_model(TYPE, EPOCH_SIZE):
  # manually set random seed
  torch.backends.cudnn.deterministic = True
  torch.manual_seed(SEED)

  # set gpu device
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  net = CNN().double().to(device)

  # pick the criterion and optimizer
  criterion = nn.MultiLabelSoftMarginLoss()
  optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

  print("Learning rate %.4f, batch size %d, node size %d, kernal size %d" % (LEARNING_RATE, BATCH_SIZE, NODE_SIZE, KERNAL_SIZE))

  # initialization
  train_acc_list = []
  val_acc_list = []
  test_acc_list = []
  accuray = 0

  # start to train with epoches
  for epoch in range(EPOCH_SIZE):
    running_loss = 0.0
    train_total = 0
    train_correct = 0
    valid_total = 0
    valid_correct = 0
    test_total = 0
    test_correct = 0

    # for the training dataset
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      optimizer.zero_grad()
      outputs = net(inputs)
      train_total += labels.size(0)
      train_correct += (torch.max(outputs, 1)[1] == torch.max(labels, 1)[1]).sum().item()
      loss = criterion(outputs, labels)
      if TYPE == 'l0_norm':
        # add group lasso regularization
        lgl = 1e-10
        cnt = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            cnt = cnt + param.detach().nonzero().size(0)
            #cnt = cnt + len(param.detach()[param.detach() > 1e-2]) + len(param.detach()[param.detach() < -1e-2])
        loss = loss + lgl * cnt
      elif TYPE == 'l1_norm':
        # add group lasso regularization
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param, 1)
        loss = loss + lgl * regularization
      elif TYPE == 'l2_norm':
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param)
        loss = loss + lgl * regularization
      elif TYPE == 'group_lasso':
        # add group lasso regularization
        lgl = 0.000001
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              regularization = regularization + torch.norm(param[i,:,:])
        loss = loss + lgl * regularization
      elif TYPE == 'l1_group_lasso':
        lgl = 0.000001
        alpha = 0.90
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            lasso_regularization = lasso_regularization + torch.norm(param, 1)
        loss = loss + (1-alpha) * lgl * group_lasso_regularization + alpha * lgl * lasso_regularization
      elif TYPE == 'l0_group_lasso':
        l0 = 1e-8
        lg = 0.4*1e-4
        cnt = torch.tensor([0]).cuda(0)
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            cnt += param.detach().nonzero().size(0)
        loss = loss + lg * group_lasso_regularization + l0 * cnt
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    # for the validation dataset
    for data in validloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      valid_total += labels.size(0)
      valid_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # for the test dataset
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # obtain the results for training, validation, test dataset
    train_acc = 100 * train_correct / train_total
    valid_acc = 100 * valid_correct / valid_total
    test_acc = 100 * test_correct / test_total
    train_acc_list.append(train_acc)
    val_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
    print("epoch %d, loss %.3f, train acc %.2f%%, valid acc %.2f%%, test acc %.2f%%" % (epoch+1, running_loss, train_acc, valid_acc, test_acc))
    
    # save the best model
    if valid_acc > accuray:
      accuray = valid_acc
      torch.save(net, PATH + '/model/' + TYPE + str(k) + ".ptl")
      torch.jit.save(torch.jit.script(net), PATH + '/model/' + TYPE + str(k) + "_git.ptl")
    
  return train_acc_list, val_acc_list, test_acc_list

# Results for CNN Model

In [ ]:
TYPE = 'no_penalty'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l0_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l1_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l2_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l1_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [18]:
TYPE = 'l0_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)
print(train_acc)
print(valid_acc)
print(test_acc)
file_name = 'data'+str(k)+'.txt'
with open(file_name, 'w') as f:
    for i in train_acc:
      f.write("%f " % i)
    f.write('\n')
    for i in valid_acc:
      f.write("%f " % i)
    f.write('\n')
    for i in test_acc:
      f.write("%f " % i)
    f.write('\n')

Learning rate 0.0002, batch size 32, node size 128, kernal size 10
epoch 1, loss 431.656, train acc 62.46%, valid acc 78.19%, test acc 78.80%
epoch 2, loss 421.160, train acc 82.19%, valid acc 85.03%, test acc 85.95%
epoch 3, loss 418.079, train acc 86.76%, valid acc 87.62%, test acc 87.93%
epoch 4, loss 416.500, train acc 88.80%, valid acc 87.79%, test acc 88.11%
epoch 5, loss 415.645, train acc 89.68%, valid acc 87.01%, test acc 87.30%
epoch 6, loss 414.863, train acc 90.65%, valid acc 89.91%, test acc 90.65%
epoch 7, loss 414.207, train acc 91.41%, valid acc 89.52%, test acc 90.09%
epoch 8, loss 413.810, train acc 91.87%, valid acc 90.40%, test acc 91.45%
epoch 9, loss 413.425, train acc 92.19%, valid acc 90.86%, test acc 91.83%
epoch 10, loss 413.131, train acc 92.61%, valid acc 90.84%, test acc 91.80%
epoch 11, loss 412.816, train acc 92.93%, valid acc 91.25%, test acc 92.09%
epoch 12, loss 412.559, train acc 93.29%, valid acc 91.25%, test acc 92.51%
epoch 13, loss 412.509, train 

# Results after Pruning the above Models

In [22]:
PRUNE_THRESHOLD = 0.01

class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
      return torch.abs(tensor) > self.threshold

PATHS = { 'No penalty - 0 128 0.0001':     PATH + '/model/final/lr0.0001/no_penalty0.ptl',
      'l0 norm - 0 128 0.0001':       PATH + '/model/final/lr0.0001/l0_norm0.ptl',
      'l1 norm - 0 128 0.0001':       PATH + '/model/final/lr0.0001/l1_norm0.ptl',
      'l2 norm - 0 128 0.0001':       PATH + '/model/final/lr0.0001/l2_norm0.ptl',
      'group lasso - 0 128 0.0001':     PATH + '/model/final/lr0.0001/group_lasso0.ptl',
      'l1 group lasso - 0 128 0.0001':   PATH + '/model/final/lr0.0001/l1_group_lasso0.ptl',
      'l0 group lasso - 0 128 0.0001 (*)': PATH + '/model/final/lr0.0001/l0_group_lasso0.ptl',
      'l0 group lasso - 0 128 0.00005':   PATH + '/model/final/lr0.00005/l0_group_lasso0.ptl',
      'l0 group lasso - 0 128 0.00015':   PATH + '/model/final/lr0.00015/l0_group_lasso0.ptl',
      'l0 group lasso - 0 128 0.0002':   PATH + '/model/final/lr0.0002/l0_group_lasso0.ptl',
      'l0 group lasso - 0 128 0.00001':   PATH + '/model/final/lr0.00001/l0_group_lasso0.ptl',
      'l0 group lasso - 0 256 0.0001':   PATH + '/model/final/256node/l0_group_lasso0.ptl',
      'l0 group lasso - 0 64 0.0001':   PATH + '/model/final/64node/l0_group_lasso0.ptl',
      'l0 group lasso - 1 128 0.0001':   PATH + '/model/final/lr0.0001/l0_group_lasso1.ptl',
      'l0 group lasso - 2 128 0.0001':   PATH + '/model/final/lr0.0001/l0_group_lasso2.ptl',
      'l0 group lasso - 3 128 0.0001':   PATH + '/model/final/lr0.0001/l0_group_lasso3.ptl',
      'l0 group lasso - 4 128 0.0001':   PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',
    }

for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net = torch.load(PATHS[name])

  # display the results before compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% before compression' % (test_total, test_acc))

  # prune the model
  parameters_to_prune = []
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      parameters_to_prune.append((child, "weight"))
  prune.global_unstructured(parameters_to_prune, pruning_method=ThresholdPruning, threshold=PRUNE_THRESHOLD)

  # calculate the sparsity
  total_weight = 0
  total_nonzero = 0
  for name, child in net.features.named_children():
    if int(name) % 2 == 0:
      total_weight += torch.numel(child.weight)
      total_nonzero += torch.count_nonzero(child.weight)
  print('Sparity for the compressed model: %.2f %%' % (100*float(total_nonzero / total_weight)))

  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))


Here are the results for No penalty - 0 128 0.0001:
Accuracy of the network on the 6072 test data: 93.15 % before compression
Sparity for the compressed model: 69.27 %
Accuracy of the network on the 6072 test data: 93.07 % after compression

Here are the results for l0 norm - 0 128 0.0001:
Accuracy of the network on the 6072 test data: 93.15 % before compression
Sparity for the compressed model: 69.27 %
Accuracy of the network on the 6072 test data: 93.07 % after compression

Here are the results for l1 norm - 0 128 0.0001:
Accuracy of the network on the 6072 test data: 95.22 % before compression
Sparity for the compressed model: 1.46 %
Accuracy of the network on the 6072 test data: 95.29 % after compression

Here are the results for l2 norm - 0 128 0.0001:
Accuracy of the network on the 6072 test data: 92.08 % before compression
Sparity for the compressed model: 65.32 %
Accuracy of the network on the 6072 test data: 92.09 % after compression

Here are the results for group lasso - 0 1